In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM


data = pd.read_csv('/.csv/bitcoin_csv')

# Preprocess the data
prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

# Define a function to create input features and target values
def create_dataset(prices, time_step=1):
    dataX, dataY = [], []
    for i in range(len(prices)-time_step-1):
        a = prices[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(prices[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

time_step = 60

# Create the input features and target values
trainX, trainY = create_dataset(scaled_prices, time_step)

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(trainX, trainY, epochs=100, batch_size=64)


last_month_prices = scaled_prices[-time_step:, :]
next_month_predictions = []

for _ in range(30):
    input_data = np.expand_dims(last_month_prices, axis=0)
    predicted_price = model.predict(input_data)[0, 0]
    next_month_predictions.append(predicted_price)

    # Shift the input data by one day and update the last month prices
    last_month_prices = np.roll(last_month_prices, -1)
    last_month_prices[-1] = predicted_price

# Inverse scale the predictions
next_month_predictions = np.array(next_month_predictions).reshape(-1, 1)
next_month_predictions = scaler.inverse_transform(next_month_predictions)

# Print the predicted prices for the next month
for i, price in enumerate(next_month_predictions, 1):
    print(f"prediction {i}: Predicted Price: {price[0]}")


Epoch 1/100
47/47 [==============================] - 13s 143ms/step - loss: 0.0178
Epoch 2/100
47/47 [==============================] - 5s 108ms/step - loss: 0.0014
Epoch 3/100
47/47 [==============================] - 6s 134ms/step - loss: 0.0012
Epoch 4/100
47/47 [==============================] - 5s 114ms/step - loss: 9.9233e-04
Epoch 5/100
47/47 [==============================] - 6s 125ms/step - loss: 9.3043e-04
Epoch 6/100
47/47 [==============================] - 6s 119ms/step - loss: 7.9971e-04
Epoch 7/100
47/47 [==============================] - 5s 110ms/step - loss: 8.9913e-04
Epoch 8/100
47/47 [==============================] - 6s 134ms/step - loss: 7.1004e-04
Epoch 9/100
47/47 [==============================] - 5s 108ms/step - loss: 7.0293e-04
Epoch 10/100
47/47 [==============================] - 6s 134ms/step - loss: 6.1016e-04
Epoch 11/100
47/47 [==============================] - 5s 109ms/step - loss: 5.4050e-04
Epoch 12/100
47/47 [==============================] - 6s 134ms/